https://hd.stheadline.com/news/realtime/fin/2391104/%E5%8D%B3%E6%99%82-%E9%87%91%E8%9E%8D-%E6%B8%A3%E6%89%93%E4%BF%A1%E7%94%A8%E5%8D%A1%E7%88%86%E5%A4%A7%E8%A6%8F%E6%A8%A1%E7%9B%9C%E7%94%A8-%E5%AE%A2%E6%9C%8D%E5%94%94%E9%80%9A-%E6%89%8B%E6%A9%9F%E6%AD%BBApp-%E6%B8%A3%E6%89%93%E6%99%9A%E4%B8%8A%E6%94%B9%E5%AE%89%E6%8E%92-%E5%B0%87%E4%B8%BB%E5%8B%95%E9%80%80%E6%AC%BE-%E7%84%A1%E8%AD%89%E6%93%9A%E8%B3%87%E6%96%99%E5%A4%96%E6%B4%A9

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import sys
import time
import requests
from itertools import cycle
from datetime import datetime
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchWindowException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import undetected_chromedriver as uc
from datetime import timedelta, date, datetime

In [2]:
#functions
def click_thread(idx):
    #click thread
    auto_xpath = f'//*[@id="leftPanel"]/div[2]/div[{idx + 1}]/div/div[2]/a[1]'
    try:
        driver.find_element(By.XPATH, auto_xpath).click()
        #print(idx)
    except:
        #print("skip")
        pass
    time.sleep(0.3)

In [3]:
#Open pandas
df_threads = pd.DataFrame(columns=["Thread UID",
                                   "Keyword",
                                   "Reference Bank",
                                   "Fetch Datetime",
                                   "Thread Author",
                                   "Thread Lastest Comment Time",
                                   "Thread Title",
                                   "Thread Number of Page",
                                   "Thread Theme",
                                   "Thread Likes",
                                   "Thread Dislike"])

df_comments = pd.DataFrame(columns=["Thread UID",
                                    "Comment UID",
                                    "Fetch Datetime",
                                    "Comment Floor Number",
                                    "Comment Author",
                                    "Comment Date",
                                    "Comment Time",
                                    "Comment Likes",
                                    "Comment Dislike",
                                    "Comment Replies",
                                    "Comment Context"])   



df_keyword = pd.read_excel('LIHKG_Keyword.xlsx', index_col=None)
keyword_list = df_keyword["Keyword"].values.tolist()
ref_bank_list = df_keyword["Reference Bank"].values.tolist()
df_keyword

,Keyword,Reference Bank
0,渣打信用卡,Standard Chartered
1,揸兜信用卡,Standard Chartered


In [4]:
#set driver to uc driver
driver = uc.Chrome()

#starts of the loop

for row, keyword in enumerate(keyword_list):
    ref_bank = ref_bank_list[row]
    keyword_uid = "LI-"+str(row).zfill(8)
    
    #get link
    driver.get(f"https://lihkg.com/search?q={keyword}&sort=desc_create_time&type=thread")
    time.sleep(3)
    
    #get html info
    html = driver.page_source
    soup = BeautifulSoup(html)
    
    #click left panel for scrolling
    driver.find_element(By.XPATH, '//*[@id="leftPanel"]/div[1]/ul/li[2]/a').click()

    #scroll to the end (leftpanel)
    while True:
        html = driver.page_source
        soup = BeautifulSoup(html)
        checker = soup.find_all("div",{"class":"_21IQKhlBjN2jlHS_TVgI3l"})
        try:
            if len(checker) == previous_checker:
                break
        except:
            pass
        
        previous_checker = len(checker)
        
        try:
            driver.find_element(By.CLASS_NAME, '_33r1FGqGJZF-fM1VZm7mhN').text
            time.sleep(2)
            break
        except:
            pass

        for i in range(20):
            ActionChains(driver)\
                .send_keys(" ")\
                .perform()
    
    #get latest html info
    html = driver.page_source
    soup = BeautifulSoup(html)
    
    #get thread html info
    threads = soup.find("div",{"class":"qoAmEqNpZRLf2KVKZ8DsC"})
    for thread_idx, i in enumerate(threads):
        #click thread
        click_thread(thread_idx)
        thread_uid = keyword_uid + str(thread_idx).zfill(8)
        
        #get thread info
        try:
            try:
                thread_author = i.find("span", {"class":"CxY4XDSSItTeLVg0cKCN0 A0jheqYUBHNW93KykXKEH"}).text
            except:
                thread_author = i.find("span", {"class":"CxY4XDSSItTeLVg0cKCN0 jj_3ZDzjtPixL1b2KTcpS"}).text

            thread_lastest_comment_time = i.find("span", {"class":"_37XwjAqVHtjzqzEtybpHrU"}).text
            #check day or hr or month function

            thread_title = i.find("span", {"class":"_20jopXBFHNQ9FUbcGHLcHH"}).text
            thread_page = i.find("div", {"class":"_26oEXjfUS_iHzbxYcZE6bD"}).text.split(" ")[0]
            thread_theme = i.find_all("a")[1].text
            thread_total_like = driver.find_elements(By.CLASS_NAME, '_8_NT40G-QNQzcSSTrRXAD')[0].get_attribute('data-score')
            thread_total_dislike = driver.find_elements(By.CLASS_NAME, '_8_NT40G-QNQzcSSTrRXAD')[1].get_attribute('data-score')

        except:

            pass
        
        #current time
        fetch_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        #save thread into to dataframe
        df_threads.loc[len(df_threads)]= [thread_uid,
                                          keyword,
                                          ref_bank,
                                          fetch_datetime,
                                          thread_author, 
                                          thread_lastest_comment_time,
                                          thread_title,
                                          int(thread_page),
                                          thread_theme, 
                                          int(thread_total_like), 
                                          int(thread_total_dislike)]

        #get the latest page info
        html = driver.page_source
        soup = BeautifulSoup(html)

        #loop to the end of pages
        for click in range(int(thread_page)):
            try:
                find_f5 = driver.find_element(By.CLASS_NAME, '_1PhR8JHkMcET5QT2PtCA3T')
                actions = ActionChains(driver)
                actions.move_to_element(find_f5).perform()
                time.sleep(1)
            except:
                pass

        #get the latest page info
        html = driver.page_source
        soup = BeautifulSoup(html)
        
        #start of fetching comments data
        all_comment = soup.find_all("div",{"class":"_36ZEkSvpdj_igmog0nluzh"})
        for comment_idx , comment in enumerate(all_comment):
            
            comment_uid = thread_uid + str(comment_idx).zfill(8)
            
            comment_floor_number = comment.find_all("span")[0].text
            comment_author = comment.find("span",{"class":"ZZtOrmcIRcvdpnW09DzFk"}).text

            #incase the comment is folded due to not enough info
            try:
                comment_datetime = comment.find("span",{"class":"Ahi80YgykKo22njTSCzs_"})
                new_comment_datetime = str(comment_datetime)[66:85]
                comment_date = new_comment_datetime.split(" ")[0]
                comment_time = new_comment_datetime.split(" ")[1]

                comment_like = comment.find_all("label")[1].text
                comment_dislike = comment.find_all("label")[3].text
            except:
                comment_datetime = ""
                comment_date = ''
                comment_time = ''
                comment_like = 0
                comment_dislike = 0

            try:
                comment_replies = comment.find_all("label")[5].text
            except:
                comment_replies = 0

            try:
                comment_context = comment.find("div",{"class":"_2cNsJna0_hV8tdMj3X6_gJ"}).text
            except:
                comment_context = ''
            
            #current time
            fetch_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            df_comments.loc[len(df_comments)]= [thread_uid,
                                                 comment_uid,
                                                 fetch_datetime,
                                                 comment_floor_number,
                                                 comment_author,
                                                 comment_date,
                                                 comment_time,
                                                 int(comment_like),
                                                 int(comment_dislike), 
                                                 int(comment_replies), 
                                                 comment_context]


In [5]:
df_threads

,Thread UID,Keyword,Reference Bank,Fetch Datetime,Thread Author,Thread Lastest Comment Time,Thread Title,Thread Number of Page,Thread Theme,Thread Likes,Thread Dislike
0,LI-0000000000000000,渣打信用卡,Standard Chartered,2023-01-02 02:23:57,GME散戶（暴發）,13 小時前,突發！張渣打信用卡俾人碌左萬鳩幾蚊！！,12,財經台,680,21
1,LI-0000000000000001,渣打信用卡,Standard Chartered,2023-01-02 02:24:17,一蓑煙雨,23 日前,渣打：近日信用卡可疑交易事件中，只有1%帳戶受影響，下星期一起陸續安排退款，並無證據顯示銀行...,2,時事台,3,18
2,LI-0000000000000002,渣打信用卡,Standard Chartered,2023-01-02 02:24:21,習慣咗就冇事,23 日前,點解渣打ATM入錢冇得即時更新信用卡餘額,1,財經台,2,4
3,LI-0000000000000003,渣打信用卡,Standard Chartered,2023-01-02 02:24:23,芝麻仁冇蘑菇,24 日前,渣打：客戶信用卡被盜用毋須責任 呼籲不用急於致電查詢,1,財經台,1,1
4,LI-0000000000000004,渣打信用卡,Standard Chartered,2023-01-02 02:24:25,毒菇天下,24 日前,渣打信用卡疑被集體盜用 熱線長期唔通全城怒「虎」！ 男苦主被盜碌逾$17000報警求助！｜C...,1,財經台,3,0
5,LI-0000000000000005,渣打信用卡,Standard Chartered,2023-01-02 02:24:27,藍色幽默,23 日前,渣打信用卡部門職員，你問我答,1,創意台,3,8
6,LI-0000000000000006,渣打信用卡,Standard Chartered,2023-01-02 02:24:28,一蓑煙雨,25 日前,渣打信用卡疑被盜用 消委會昨日至今接4宗投訴,1,時事台,0,0
7,LI-0000000000000007,渣打信用卡,Standard Chartered,2023-01-02 02:24:30,鹿特丹斯巴光年,22 日前,渣打信用卡客戶懷疑被大規模盜用 官方最新回應：確定非授權交易可獲退款！,4,時事台,0,21
8,LI-0000000000000008,渣打信用卡,Standard Chartered,2023-01-02 02:24:36,鹿特丹斯巴光年,22 日前,渣打信用卡客戶懷疑被大規模盜用 官方最新回應：確定非授權交易可獲退款！,4,時事台,0,21
9,LI-0000000000000009,渣打信用卡,Standard Chartered,2023-01-02 02:24:42,有冇人打波,25 日前,[多謝你班恐懼撚] 渣打信用卡,1,時事台,10,21


In [6]:
df_comments

,Thread UID,Comment UID,Fetch Datetime,Comment Floor Number,Comment Author,Comment Date,Comment Time,Comment Likes,Comment Dislike,Comment Replies,Comment Context
0,LI-0000000000000000,LI-000000000000000000000000,2023-01-02 02:24:16,#1,GME散戶（暴發）,2022年12月9日,09:37:13,680,21,0,\n\n\n\n\n\n覆完之後入app 睇\n我根本冇呢張尾數既卡\n做乜鳩 咩事？？
1,LI-0000000000000000,LI-000000000000000000000001,2023-01-02 02:24:16,#2,洪為民請命,2022年12月9日,09:38:01,75,308,15,坐撚咗兩日？無睇新聞？
2,LI-0000000000000000,LI-000000000000000000000002,2023-01-02 02:24:16,#3,99912345,2022年12月9日,09:39:06,83,0,0,無事 呢幾日算係正常
3,LI-0000000000000000,LI-000000000000000000000003,2023-01-02 02:24:16,#4,那日的笑容,2022年12月9日,09:55:38,77,20,10,呢鋪渣打會唔會破產清盤?
4,LI-0000000000000000,LI-000000000000000000000004,2023-01-02 02:24:16,#5,奧斯卡PS3,2022年12月9日,09:55:42,219,3,1,根本冇呢張尾數既卡
...,...,...,...,...,...,...,...,...,...,...,...
2722,LI-0000000100000002,LI-000000010000000200000762,2023-01-02 02:29:11,#790,安心掃閪,2022年7月9日,"19:56:02""",0,0,0,邊間銀行黎架
2723,LI-0000000100000002,LI-000000010000000200000763,2023-01-02 02:29:11,#791,賈雨村,2022年7月9日,"20:00:28""",0,0,0,行山銀行
2724,LI-0000000100000002,LI-000000010000000200000764,2023-01-02 02:29:11,#792,賈雨村,2022年7月20日,19:47:53,0,0,0,東張又黎啦
2725,LI-0000000100000002,LI-000000010000000200000765,2023-01-02 02:29:11,#793,呂爵安全套,,,0,0,0,


In [7]:
#output data
df_threads.to_excel('LIHKG_Thread_Level.xlsx',encoding= 'utf-8-sig',index = None) 
df_comments.to_excel('LIHKG_Comment_Level.xlsx',encoding= 'utf-8-sig',index = None) 

C:\Users\User\anaconda3\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
